# Importing the necessary Libraries

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Loading and Preprocessing the Video Data

In [3]:
def load_videos(video_dir, label):
    video_data = []
    labels = []
    for video_name in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (112, 112))  # Resize to a fixed size
            frames.append(frame)
            if len(frames) == 16:  # Use 16 frames per video
                break
        cap.release()
        frames = np.array(frames)
        if frames.shape == (16, 112, 112, 3):  # Ensure correct shape
            video_data.append(frames)
            labels.append(label)
    return np.array(video_data), np.array(labels)

real_videos, real_labels = load_videos('separated_dataset/Real/', label=0)
fake_videos, fake_labels = load_videos('separated_dataset/Fake/', label=1)

# Combine the datasets
X = np.concatenate((real_videos, fake_videos), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)

# Convert labels to categorical format
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y, 2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building

In [4]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Input

model = Sequential()
model.add(Input(shape=(16, 112, 112, 3)))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 14, 110, 110, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 7, 55, 55, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 5, 53, 53, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 2, 26, 26, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,361,858 (169.23 MB)

 Trainable params: 44,361,858 (169.23 MB)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [5]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=8)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 328ms/step - accuracy: 0.7248 - loss: 369.1166 - val_accuracy: 0.7656 - val_loss: 0.6553
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 328ms/step - accuracy: 0.8305 - loss: 0.6136 - val_accuracy: 0.7969 - val_loss: 0.5848
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 345ms/step - accuracy: 0.8297 - loss: 0.5277 - val_accuracy: 0.7969 - val_loss: 0.5521
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 332ms/step - accuracy: 0.8228 - loss: 0.4824 - val_accuracy: 0.7969 - val_loss: 0.5494
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 329ms/step - accuracy: 0.8277 - loss: 0.4729 - val_accuracy: 0.7969 - val_loss: 0.5092
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 335ms/step - accuracy: 0.8381 - loss: 0.4432 - val_accuracy: 0.7969 - val_loss: 0.5179
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 335ms/step - accuracy: 0.8239 - loss: 0.4703 - val_accuracy: 0.7969 - val_loss: 0.5062
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 336ms/step - accuracy: 0.8391 - loss: 0.4341 - val_ac

# Evaluating the Model

In [6]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.7578 - loss: 0.5567
Test Accuracy: 75.00%


# Making Predictions on new videos

In [7]:
def predict_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (112, 112))
        frames.append(frame)
        if len(frames) == 16:
            break
    cap.release()
    frames = np.array(frames)
    if frames.shape == (16, 112, 112, 3):
        frames = np.expand_dims(frames, axis=0)
        prediction = model.predict(frames)
        return np.argmax(prediction)

# Example usage
video_path = 'test_videos/ahjnxtiamx.mp4'
prediction = predict_video(video_path, model)
print("Prediction: ", "Fake" if prediction == 1 else "Real")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Prediction:  Fake
